## vLLM

## 初始環境設定

In [ ]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

## 安裝套件
安裝完成後建議, 點選上方選單, 直接階段->重新啟動工作階段, 確保 library重置

In [ ]:
!pip install https://github.com/vllm-project/vllm/releases/download/v0.3.0/vllm-0.3.0+cu118-cp310-cp310-manylinux1_x86_64.whl -q
!pip install --upgrade xformers==0.0.23.post1 torch==2.1.2 --index-url https://download.pytorch.org/whl/cu118 -q

## 確認CUDA版本, 以及否能使用GPU
若無gpu 請點選右側->已連線->變更執行階段類型->T4 Gpu

In [ ]:
!nvidia-smi
import torch
print(torch.__version__)  #注意是双下划线
torch.cuda.is_available()

## MEMO

In [ ]:
import sys
#sys.executable="/usr/bin/python3.10"
#!{sys.executable} -V

#from pprint import pprint
#pprint(sys.path)
#sys.path.append(’需要引用模块的地址')


###　Model Vllm

In [ ]:
!python3 -m vllm.entrypoints.api_server \
    --model /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --swap-space 16 \
    --disable-log-requests --host 0.0.0.0 --port 8080 --max-num-seqs 256

##　Model Openai

In [ ]:
!python3 -m vllm.entrypoints.openai.api_server \
--model /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --swap-space 16 \
--disable-log-requests --host 0.0.0.0 --port 5001 --max-num-seqs 20 --served-model-name llama-2

## CURL

In [ ]:
!curl http://gpn3001:5001/v1/models

In [ ]:
%%bash
curl http://gpn3001:5001/v1/completions \
-H "Content-Type: application/json" \
-d '{
"model": "llama-2",
"prompt": "San Francisco is a",
"max_tokens": 70,
"temperature": 0
}'

In [ ]:
%%bash
curl http://gpn3001:5001/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
"model": "llama-2",
"messages": [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": "什麼是聯邦式學習?"}
]
}'

## Singularity　Model Openai
change kernel to Pytohn 3 (ipykernel)

In [ ]:
!/work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity exec --nv -C \
-B /work -B /work/u00cjz00/os/ubuntu/bin:/usr/ubuntu_bin \
-B /work/g00cjz00/libraryFolder/P-3.10-work-genai_cuda_11.8.0-cudnn8-runtime-ubuntu22.04/local/lib:/home/g00cjz00/.local/lib \
-B /work/g00cjz00/libraryFolder/P-3.10-work-genai_cuda_11.8.0-cudnn8-runtime-ubuntu22.04/local/bin:/home/g00cjz00/.local/bin \
/work/u00cjz00/nvidia/cuda_11.8.0-cudnn8-runtime-ubuntu22.04.sif \
bash -c 'export PATH=$PATH:$HOME/.local/bin; \
python3 -m vllm.entrypoints.openai.api_server \
--model /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --swap-space 16 \
--disable-log-requests --host 0.0.0.0 --port 5001 --max-num-seqs 20 --served-model-name llama-2'

## SLURM SingularityModel Openai
change kernel to Pytohn 3 (ipykernel)

In [ ]:
%%bash
cat << \EOF >  demo.slurm
#!/work/u00cjz00/binary/bash5.0/bin/bash
#SBATCH -A GOV109189                                                    ### project number, Example MST109178
#SBATCH -J _t2vllm_                                                     ### Job name, Exmaple jupyterlab
#SBATCH -p gp4d                                                         ### Partition Name, Example ngs1gpu
#SBATCH --nodes=1                                                       ### Nodes, Default 1, node number
#SBATCH --ntasks-per-node=1                                             ### Tasks, Default 1, per node tasks
#SBATCH -c 4                                                            ### Cores assigned to each task, Example 4
#SBATCH --gres=gpu:1                                                    ### GPU number, Example gpu:1
#SBATCH --time=0-1:00:00                                                ### Runnung time, days-hours:minutes:seconds or hours:minutes:seconds
#SBATCH -o genai_%j.out                                                 ### Log folder, Here %j is job ID
#SBATCH -e genai_%j.err                                                 ### Log folder, Here %j is job ID

/work/opt/ohpc/Taiwania3/libs/singularity/3.10.2/bin/singularity exec --nv -C \
-B /work -B /work/u00cjz00/os/ubuntu/bin:/usr/ubuntu_bin \
-B /work/g00cjz00/libraryFolder/P-3.10-work-genai_cuda_11.8.0-cudnn8-runtime-ubuntu22.04/local/lib:/home/g00cjz00/.local/lib \
-B /work/g00cjz00/libraryFolder/P-3.10-work-genai_cuda_11.8.0-cudnn8-runtime-ubuntu22.04/local/bin:/home/g00cjz00/.local/bin \
/work/u00cjz00/nvidia/cuda_11.8.0-cudnn8-runtime-ubuntu22.04.sif \
bash -c 'export PATH=$PATH:$HOME/.local/bin; \
python3 -m vllm.entrypoints.openai.api_server \
--model /work/u00cjz00/slurm_jobs/github/models/Llama-2-7b-chat-hf --swap-space 16 \
--disable-log-requests --host 0.0.0.0 --port 5001 --max-num-seqs 20 --served-model-name llama-2'

EOF

sbatch  demo.slurm

## CHECK SLURM

In [ ]:
!squeue -u $(whoami)

In [ ]:
!cat genai_569512.out
!cat genai_569512.err

## CURL

In [ ]:
!curl http://gn0901:5001/v1/models

In [ ]:
%%bash
curl http://gn0901:5001/v1/completions \
-H "Content-Type: application/json" \
-d '{
"model": "llama-2",
"prompt": "San Francisco is a",
"max_tokens": 70,
"temperature": 0
}'

In [ ]:
%%bash
curl http://gn0901:5001/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
"model": "llama-2",
"messages": [
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": "什麼是聯邦式學習?"}
]
}'